In [1]:
# setup
import time, os, calendar, sys
import envkey
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
import seaborn as sns
from sqlalchemy.engine import create_engine
import math
import datetime

# allow importing modules from ../..
sys.path.insert(1, os.path.join(sys.path[0], '../..'))

register_matplotlib_converters()
sns.set(rc={'figure.figsize':(11, 4)})

ANALYTICS_MYSQL_ENDPOINT = os.getenv('ANALYTICS_MYSQL_ENDPOINT')
ANALYTICS_MYSQL_PASSWORD = os.getenv('ANALYTICS_MYSQL_PASSWORD')

if not (ANALYTICS_MYSQL_ENDPOINT and ANALYTICS_MYSQL_PASSWORD):
    raise Exception('ANALYTICS_MYSQL credentials not found')

def get_engine(game):
    database_name = '%s_upcache' % game
    return create_engine(f'mysql://analytics1:{ANALYTICS_MYSQL_PASSWORD}@{ANALYTICS_MYSQL_ENDPOINT}/{database_name}', connect_args={'connect_timeout': 10})

def get_quarter_time_boundaries(timestamp):
    q = {}
    q['start'] = 1640217600
    q['end'] = q['start'] + (86400 * 7 * 14) # 14 weeks, 13 weeks for the quarter, plus one trailing week
    while not (timestamp >= q['start'] and timestamp <= q['end']):
        q['start'] = q['end']
        q['end'] = q['start'] + (86400 * 7 * 14)
    return q

def get_maintenance_okr_receipts_per_spend_sql(game, start_time, end_time, level):
    return """SELECT timeline.day AS `day`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d) AS `dollars`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 AND rolling.day <= timeline.day
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucks`,
              (SELECT SUM(rolling.usd_receipts_cents)
              FROM `%s_credits_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d) AS `dollarsprev`,
              (SELECT SUM(rolling.total_price)
              FROM `%s_store_daily_summary` AS rolling
              WHERE rolling.day >= timeline.day - 6*86400 - 90*86400 AND rolling.day <= timeline.day - 90*86400
              AND rolling.townhall_level >= %d
              AND rolling.currency = 'gamebucks') AS `gamebucksprevqtr`
              FROM (SELECT sess.day AS day
              FROM `%s_sessions_daily_summary` AS sess
              WHERE sess.day >= %d AND sess.day < %d
              GROUP BY sess.day ORDER BY sess.day ASC) AS timeline LIMIT 999999;""" % (game, level, game, level, game, level, game, level, game, start_time, end_time)

def get_maintenance_okr_dollar_receipts_sql(game, lookup_time, level):
    return """SELECT SUM(usd_receipts_cents) as cents
              FROM `%s_credits_daily_summary`
              WHERE day >= %d - 6*86400 AND day <= %d
              AND townhall_level >= %d""" % (game, lookup_time, lookup_time, level)

def get_maintenance_okr_gamebucks_sql(game, lookup_time, level):
    return """SELECT SUM(total_price) as gamebucks
              FROM `%s_store_daily_summary`
              WHERE day >= %d - 6*86400 AND day <= %d
              AND townhall_level >= %d
              AND currency = 'gamebucks'; """ % (game, lookup_time, lookup_time, level)

def get_maintenance_okr_timeline_sql(game, start_time, end_time):
    return """SELECT day
              FROM %s_sessions_daily_summary
              WHERE day >= %d AND day < %d
              GROUP BY day ORDER BY day ASC LIMIT 9999;""" % (game, start_time, end_time)

time_now = int(time.time())
quarter_boundaries = get_quarter_time_boundaries(time_now)
start_time = quarter_boundaries['start']
end_time = quarter_boundaries['end']

# prepare maintenance OKR data

engine = get_engine('tr')
#maintenance_okr_receipts_per_spend_sql = get_maintenance_okr_receipts_per_spend_sql('tr',start_time,end_time,5)
#maintenance_okr_receipts_per_spend_data  = pd.read_sql(maintenance_okr_receipts_per_spend_sql, engine)

print(f"Dashboard updated %s" % time.strftime('%a, %d %b %Y at %H:%M:%S UTC', time.gmtime()))

Dashboard updated Sun, 17 Apr 2022 at 19:54:31 UTC


In [10]:
maintenance_okr_timeline_sql = get_maintenance_okr_timeline_sql('tr', start_time, end_time)
maintenance_okr_timeline_data = pd.read_sql(maintenance_okr_timeline_sql, engine)

maintenance_okr_receipts_per_spend_data = []
for i, day in enumerate(maintenance_okr_timeline_data['day']):
    result = {'day': day}
    maintenance_okr_dollar_receipts_sql = get_maintenance_okr_dollar_receipts_sql('tr',day,5)
    maintenance_okr_dollar_receipts_data = pd.read_sql(maintenance_okr_dollar_receipts_sql, engine)
    maintenance_okr_gamebucks_sql = get_maintenance_okr_gamebucks_sql('tr',day,5)
    maintenance_okr_gamebucks_data = pd.read_sql(maintenance_okr_gamebucks_sql, engine)
    result['dollars'] = maintenance_okr_dollar_receipts_data['cents'][0]# / 100
    result['gamebucks'] = maintenance_okr_gamebucks_data['gamebucks'][0]
    maintenance_okr_receipts_per_spend_data.append(result)

# maintenance OKR display
boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries = get_quarter_time_boundaries(time_now)
diplay_boundaries['start'] = diplay_boundaries['start'] + 86400*7
print('TR USD Receipts per gamebuck')
header = "{0:12} {1:8}".format('Week', 'Receipts/Gamebuck')
print(header)
for data in maintenance_okr_receipts_per_spend_data:
    day = data['day']
    if day < boundaries['start'] or day > boundaries['end']: continue
    output_date = datetime.datetime.utcfromtimestamp(day).strftime('%Y-%m-%d')
    output_receipts = data['dollars'] / data['gamebucks']
    if day < diplay_boundaries['start'] or day > diplay_boundaries['end']: continue
    if datetime.datetime.utcfromtimestamp(day).strftime('%A') == 'Thursday':
        output_line = "{0:12} {1:8}".format(output_date, str(round(output_receipts, 2)))
        print(output_line)

TR USD Receipts per gamebuck
Week         Receipts/Gamebuck
2022-04-07   0.5     
2022-04-14   0.43    
